In [1]:
using DataFrames, CSV, JuMP, Gurobi, Plots, Random

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [3]:
function team_match()
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    N = 20 # teams
    W = 38

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = 1:W], Bin) # 1 if team i plays team j on week w, 0 otherwise

    # OBJECTIVE FUNCTION
    @objective(model, Max, sum(x[i,j,w] for i in 1:N, j in 1:N, w in 1:W))

    # CONSTRAINTS
    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = 1:W], x[i, i, w] == 0)

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = 1:W], sum(x[i, j, w] + x[j, i, w] for j in 1:N) <= 1)

    for i=1:N
        for j=1:N
            if i!=j
                # each game is against a different team, team play against each other team at most once in first half of season
                @constraint(model, sum(x[i, j, w] + x[j, i, w] for w in 1:Int(W/2)) == 1)
                
                # each game is against a different team, team play against each other team at most once in second half of season
                @constraint(model, sum(x[i, j, w] + x[j, i, w] for w in Int((W/2))+1:W) == 1)

                # every team plays every other team once at home
                @constraint(model, sum(x[i, j, w] for w in 1:W) == 1)
                
            end
        end
    end

    # no more than 2 straight home games
    @constraint(model, [i = 1:N, w = 1:W-2], sum(x[i, :, w] + x[i, :, w+1] + x[i, :, w+2]) <= 2)

    # no more than 2 straight away games
    @constraint(model, [j = 1:N, w = 1:W-2], sum(x[:, j, w] + x[:, j, w+1] + x[:, j, w+2]) <= 2)


    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)


    return value.(x)
end

team_match (generic function with 1 method)

In [4]:
x = team_match()

20×20×38 Array{Float64, 3}:
[:, :, 1] =
  0.0  -0.0  -0.0  -0.0   0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0   0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0   0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0   1.0  -0.0
  1.0  -0.0  -0.0  -0.0   0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0   0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   1.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0   1.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0   1.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0

In [5]:
function table_games_w(x, team)
    ## create w dataframes, one for each week, each dataframe contains the games for that week
    N = 20
    W = 38
    table = zeros(W,2)
    for w=1:W
        for j=1:N
            if x[team,j,w] == 1 && x[j,team,w] == 0
                table[w,1] = team
                table[w,2] = j 
            elseif x[team,j,w] == 0 && x[j,team,w] == 1
                table[w,1] = j
                table[w,2] = team
            end
        end
    end
    return table
end

table_games_w (generic function with 1 method)

In [6]:
function all_games(x)
     
    N = 20
    W = 38

    team1 = table_games_w(x,1)
    team2 = table_games_w(x,2)
    team3 = table_games_w(x,3)
    team4 = table_games_w(x,4)
    team5 = table_games_w(x,5)
    team6 = table_games_w(x,6)
    team7 = table_games_w(x,7)
    team8 = table_games_w(x,8)
    team9 = table_games_w(x,9)
    team10 = table_games_w(x,10)
    team11 = table_games_w(x,11)
    team12 = table_games_w(x,12)
    team13 = table_games_w(x,13)
    team14 = table_games_w(x,14)
    team15 = table_games_w(x,15)
    team16 = table_games_w(x,16)
    team17 = table_games_w(x,17)
    team18 = table_games_w(x,18)
    team19 = table_games_w(x,19)
    team20 = table_games_w(x,20)

    table = zeros(10,W*2)
    for w=1:W
        c = 1
        if team1[w,1] == 1
            table[c,w*2-1] = team1[w,1]
            table[c,w*2] = team1[w,2]
            c += 1
        end 
        if team2[w,1] == 2
            table[c,w*2-1] = team2[w,1]
            table[c,w*2] = team2[w,2]
            c += 1
        end
        if team3[w,1] == 3
            table[c,w*2-1] = team3[w,1]
            table[c,w*2] = team3[w,2]
            c += 1
        end
        if team4[w,1] == 4
            table[c,w*2-1] = team4[w,1]
            table[c,w*2] = team4[w,2]
            c += 1
        end
        if team5[w,1] == 5
            table[c,w*2-1] = team5[w,1]
            table[c,w*2] = team5[w,2]
            c += 1
        end
        if team6[w,1] == 6
            table[c,w*2-1] = team6[w,1]
            table[c,w*2] = team6[w,2]
            c += 1
        end
        if team7[w,1] == 7
            table[c,w*2-1] = team7[w,1]
            table[c,w*2] = team7[w,2]
            c += 1
        end
        if team8[w,1] == 8
            table[c,w*2-1] = team8[w,1]
            table[c,w*2] = team8[w,2]
            c += 1
        end
        if team9[w,1] == 9
            table[c,w*2-1] = team9[w,1]
            table[c,w*2] = team9[w,2]
            c += 1
        end
        if team10[w,1] == 10
            table[c,w*2-1] = team10[w,1]
            table[c,w*2] = team10[w,2]
            c += 1
        end
        if team11[w,1] == 11
            table[c,w*2-1] = team11[w,1]
            table[c,w*2] = team11[w,2]
            c += 1
        end
        if team12[w,1] == 12
            table[c,w*2-1] = team12[w,1]
            table[c,w*2] = team12[w,2]
            c += 1
        end
        if team13[w,1] == 13
            table[c,w*2-1] = team13[w,1]
            table[c,w*2] = team13[w,2]
            c += 1
        end
        if team14[w,1] == 14
            table[c,w*2-1] = team14[w,1]
            table[c,w*2] = team14[w,2]
            c += 1
        end
        if team15[w,1] == 15
            table[c,w*2-1] = team15[w,1]
            table[c,w*2] = team15[w,2]
            c += 1
        end
        if team16[w,1] == 16
            table[c,w*2-1] = team16[w,1]
            table[c,w*2] = team16[w,2]
            c += 1
        end
        if team17[w,1] == 17
            table[c,w*2-1] = team17[w,1]
            table[c,w*2] = team17[w,2]
            c += 1
        end
        if team18[w,1] == 18
            table[c,w*2-1] = team18[w,1]
            table[c,w*2] = team18[w,2]
            c += 1
        end
        if team19[w,1] == 19
            table[c,w*2-1] = team19[w,1]
            table[c,w*2] = team19[w,2]
            c += 1
        end
        if team20[w,1] == 20
            table[c,w*2-1] = team20[w,1]
            table[c,w*2] = team20[w,2]
            c += 1
        end
    end

    ### change table into dataframe
    df_games = DataFrame(W1_Home=table[:,1], W1_Away=table[:,2], W2_Home=table[:,3], W2_Away=table[:,4], W3_Home=table[:,5], W3_Away=table[:,6], W4_Home=table[:,7], W4_Away=table[:,8], W5_Home=table[:,9], W5_Away=table[:,10], W6_Home=table[:,11], W6_Away=table[:,12], W7_Home=table[:,13], W7_Away=table[:,14], W8_Home=table[:,15], W8_Away=table[:,16], W9_Home=table[:,17], W9_Away=table[:,18], W10_Home=table[:,19], W10_Away=table[:,20], W11_Home=table[:,21], W11_Away=table[:,22], W12_Home=table[:,23], W12_Away=table[:,24], W13_Home=table[:,25], W13_Away=table[:,26], W14_Home=table[:,27], W14_Away=table[:,28], W15_Home=table[:,29], W15_Away=table[:,30], W16_Home=table[:,31], W16_Away=table[:,32], W17_Home=table[:,33], W17_Away=table[:,34], W18_Home=table[:,35], W18_Away=table[:,36], W19_Home=table[:,37], W19_Away=table[:,38], W20_Home=table[:,39], W20_Away=table[:,40], W21_Home=table[:,41], W21_Away=table[:,42], W22_Home=table[:,43], W22_Away=table[:,44], W23_Home=table[:,45], W23_Away=table[:,46], W24_Home=table[:,47], W24_Away=table[:,48], W25_Home=table[:,49], W25_Away=table[:,50], W26_Home=table[:,51], W26_Away=table[:,52], W27_Home=table[:,53], W27_Away=table[:,54], W28_Home=table[:,55], W28_Away=table[:,56], W29_Home=table[:,57], W29_Away=table[:,58], W30_Home=table[:,59], W30_Away=table[:,60], W31_Home=table[:,61], W31_Away=table[:,62], W32_Home=table[:,63], W32_Away=table[:,64], W33_Home=table[:,65], W33_Away=table[:,66], W34_Home=table[:,67], W34_Away=table[:,68], W35_Home=table[:,69], W35_Away=table[:,70], W36_Home=table[:,71], W36_Away=table[:,72], W37_Home=table[:,73], W37_Away=table[:,74], W38_Home=table[:,75], W38_Away=table[:,76])

    return df_games
end         

all_games (generic function with 1 method)

In [7]:
all_games(x)

Row,W1_Home,W1_Away,W2_Home,W2_Away,W3_Home,W3_Away,W4_Home,W4_Away,W5_Home,W5_Away,W6_Home,W6_Away,W7_Home,W7_Away,W8_Home,W8_Away,W9_Home,W9_Away,W10_Home,W10_Away,W11_Home,W11_Away,W12_Home,W12_Away,W13_Home,W13_Away,W14_Home,W14_Away,W15_Home,W15_Away,W16_Home,W16_Away,W17_Home,W17_Away,W18_Home,W18_Away,W19_Home,W19_Away,W20_Home,W20_Away,W21_Home,W21_Away,W22_Home,W22_Away,W23_Home,W23_Away,W24_Home,W24_Away,W25_Home,W25_Away,W26_Home,W26_Away,W27_Home,W27_Away,W28_Home,W28_Away,W29_Home,W29_Away,W30_Home,W30_Away,W31_Home,W31_Away,W32_Home,W32_Away,W33_Home,W33_Away,W34_Home,W34_Away,W35_Home,W35_Away,W36_Home,W36_Away,W37_Home,W37_Away,W38_Home,W38_Away
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2.0,12.0,1.0,2.0,2.0,17.0,1.0,12.0,3.0,5.0,2.0,13.0,1.0,8.0,1.0,18.0,2.0,15.0,1.0,16.0,3.0,1.0,2.0,4.0,1.0,9.0,3.0,19.0,1.0,13.0,1.0,17.0,2.0,14.0,1.0,14.0,1.0,10.0,2.0,6.0,1.0,7.0,2.0,9.0,2.0,18.0,1.0,4.0,1.0,11.0,2.0,1.0,1.0,19.0,1.0,3.0,3.0,10.0,1.0,6.0,2.0,10.0,1.0,5.0,2.0,7.0,1.0,15.0,2.0,19.0,1.0,20.0,3.0,2.0,2.0,8.0
2,4.0,19.0,3.0,6.0,4.0,1.0,2.0,11.0,4.0,9.0,3.0,4.0,5.0,11.0,2.0,5.0,4.0,8.0,3.0,11.0,4.0,5.0,6.0,1.0,3.0,14.0,7.0,4.0,2.0,16.0,3.0,15.0,4.0,17.0,2.0,3.0,3.0,18.0,5.0,20.0,3.0,17.0,3.0,8.0,6.0,5.0,5.0,16.0,3.0,12.0,3.0,20.0,4.0,14.0,2.0,20.0,5.0,7.0,3.0,9.0,4.0,11.0,3.0,7.0,4.0,15.0,3.0,13.0,4.0,7.0,4.0,3.0,5.0,4.0,7.0,6.0
3,5.0,1.0,8.0,20.0,5.0,18.0,5.0,13.0,8.0,14.0,5.0,14.0,6.0,7.0,3.0,16.0,7.0,18.0,5.0,19.0,11.0,13.0,7.0,13.0,4.0,12.0,9.0,2.0,4.0,18.0,6.0,2.0,5.0,6.0,5.0,8.0,4.0,13.0,8.0,7.0,5.0,2.0,4.0,20.0,7.0,19.0,6.0,12.0,4.0,6.0,6.0,14.0,5.0,12.0,4.0,16.0,6.0,19.0,4.0,2.0,5.0,3.0,8.0,16.0,5.0,17.0,5.0,10.0,9.0,6.0,6.0,15.0,6.0,17.0,9.0,4.0
4,7.0,3.0,9.0,10.0,6.0,9.0,6.0,10.0,10.0,2.0,7.0,16.0,9.0,17.0,4.0,10.0,10.0,5.0,6.0,4.0,12.0,6.0,9.0,3.0,5.0,15.0,11.0,1.0,5.0,9.0,8.0,19.0,7.0,1.0,6.0,11.0,7.0,5.0,9.0,16.0,6.0,16.0,7.0,10.0,8.0,4.0,8.0,9.0,7.0,15.0,9.0,5.0,7.0,11.0,9.0,15.0,8.0,1.0,7.0,12.0,6.0,8.0,9.0,13.0,6.0,3.0,6.0,20.0,10.0,12.0,8.0,5.0,7.0,14.0,11.0,5.0
5,8.0,18.0,11.0,12.0,10.0,13.0,7.0,20.0,11.0,7.0,8.0,6.0,13.0,3.0,6.0,13.0,11.0,14.0,7.0,2.0,15.0,10.0,10.0,17.0,6.0,18.0,12.0,5.0,7.0,8.0,9.0,7.0,8.0,3.0,10.0,19.0,8.0,17.0,11.0,10.0,10.0,15.0,13.0,6.0,9.0,1.0,11.0,17.0,10.0,8.0,11.0,16.0,8.0,15.0,10.0,6.0,9.0,18.0,10.0,14.0,7.0,17.0,10.0,4.0,9.0,14.0,7.0,9.0,13.0,11.0,11.0,9.0,8.0,13.0,12.0,15.0
6,9.0,20.0,13.0,16.0,11.0,8.0,9.0,8.0,13.0,12.0,10.0,18.0,14.0,10.0,8.0,12.0,12.0,3.0,8.0,10.0,16.0,8.0,12.0,18.0,8.0,2.0,13.0,8.0,10.0,3.0,11.0,4.0,11.0,18.0,12.0,7.0,9.0,11.0,12.0,1.0,12.0,9.0,14.0,5.0,10.0,20.0,13.0,18.0,14.0,13.0,12.0,8.0,10.0,9.0,12.0,11.0,11.0,2.0,13.0,5.0,12.0,13.0,11.0,6.0,10.0,16.0,8.0,11.0,14.0,1.0,12.0,2.0,9.0,19.0,16.0,14.0
7,10.0,11.0,15.0,4.0,12.0,19.0,14.0,4.0,15.0,1.0,11.0,15.0,15.0,12.0,11.0,20.0,13.0,17.0,9.0,12.0,17.0,7.0,14.0,16.0,10.0,7.0,14.0,6.0,12.0,20.0,14.0,12.0,12.0,10.0,13.0,9.0,14.0,19.0,15.0,3.0,13.0,4.0,15.0,11.0,11.0,3.0,14.0,3.0,16.0,2.0,13.0,7.0,13.0,2.0,14.0,8.0,12.0,16.0,15.0,19.0,14.0,15.0,14.0,2.0,11.0,19.0,12.0,4.0,15.0,5.0,13.0,10.0,10.0,1.0,17.0,1.0
8,13.0,14.0,17.0,5.0,14.0,7.0,17.0,3.0,16.0,20.0,12.0,17.0,16.0,4.0,14.0,9.0,16.0,9.0,13.0,15.0,18.0,9.0,15.0,8.0,13.0,19.0,16.0,10.0,15.0,14.0,16.0,5.0,13.0,20.0,17.0,16.0,15.0,6.0,17.0,14.0,14.0,11.0,16.0,19.0,12.0,14.0,15.0,2.0,17.0,9.0,15.0,18.0,16.0,3.0,17.0,13.0,14.0,20.0,16.0,17.0,16.0,1.0,15.0,20.0,13.0,1.0,16.0,18.0,17.0,8.0,14.0

In [8]:
# C is upper triangular matrix; each entry is the cost of playing in t and then t_
function GetC(x,y)
    if x == y
        return 10
    end
    return 1/(y-x)
end

GetC (generic function with 1 method)

In [11]:
### Teams 1,2 play on weeks 5,6,7,12,13,14 on Tuesday
### Teams 3,4 play on weeks 5,6,7,12,13,14 on Wednesday
### Teams 5,6 play on weeks 5,6,7,12,13,14 on Thursday
u = zeros((20,39)) #adding week 39 just so the model can factor in the last week
z = zeros((20,39))
for i in 1:6
    for w in 1:38
        if w == 5 || w == 6 || w == 7 || w == 12 || w == 13 || w ==14 
            u[i,w] = 1.0
            if i == 1 || i == 2
                z[i,w] = 2.0
            elseif i == 3 || i == 4
                z[i,w] = 3.0
            elseif i == 5 || i == 6
                z[i,w] = 4.0
            end
        end
    end
end
z

20×39 Matrix{Float64}:
 0.0  0.0  0.0  0.0  2.0  2.0  2.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  2.0  2.0  2.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  3.0  3.0  3.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  3.0  3.0  3.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  4.0  4.0  4.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  4.0  4.0  4.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [14]:
function schedule_days(x,u,z)
    N=20
    W=38
    D=7 #playable number of days

    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 180)
    set_optimizer_attribute(model, "OutputFlag", 1)
    set_optimizer_attribute(model, "MIPGap", 1e-4)

    # VARIABLES
    @variable(model, y[i = 1:N, w = 1:W+1, d = 4:D], Bin) # 1 if team i plays on day d of week w, 0 otherwise
    #filler week W+1 to help with model

    # OBJECTIVE FUNCTION -- rest of one team doesn't depend on the other team, put that in the constraints
    @objective(model, Min, 
        sum( ( (1-u[i,w+1])*GetC(d,7+d_) + u[i,w+1]*(GetC(d,7+z[i,w+1])+GetC(z[i,w+1], d_)) )

            * (y[i,w,d]*y[i,w+1,d_])
        for i in 1:N, w in 1:W, d in 4:D, d_ in 4:D))

    # # OBJECTIVE FUNCTION
    # @objective(model, Min, 
    #     sum( ( (1-u[i,w+1])*GetC(d,7+d_) + u[i,w+1]*(GetC(d,7+z[i,w+1])+GetC(z[i,w+1], d_)) )

    #         * (
    #             (x[i,j,w]+x[j,i,w])*(x[i,k,w+1]+x[k,i,w+1])
    #         * y[i,w,d]*y[i,w+1,d_]
    #         )

    #     for i in 1:N, j in 1:N, k in 1:N, w in 1:W, d in 4:D, d_ in 4:D))

    # CONSTRAINTS
    
    # # teams who play each other on week w must play on the same day of the week
    # @constraint(model, [i = 1:N, j = 1:N, w = 1:W, d = 4:D], y[i,w,d]*y[j,w,d] >= x[i,j,w]+x[j,i,w])

    # # teams who play each other on week w must play on the same day of the week
    # @constraint(model, [i = 1:N, j = 1:N, w = 1:W], sum(y[i,w,d]*y[j,w,d]*(x[i,j,w]+x[j,i,w]) for d in 4:D) == 1)
    
    # This constraint holds only when the teams play exactly one game per week
    #for w=1:W
    #    for i=1:N
    #        for j=1:N
    #            if x[i,j,w]+x[j,i,w] == 1
    #                @constraint(model, (y[i,w,:] .== y[j,w,:]))
    #            end
    #        end
    #    end
    #end
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W, d = 4:D], y[i,w,d] >= (x[i,j,w]+x[j,i,w])*y[j,w,d])

    # filler week 39
    @constraint(model, [i = 1:N], y[i,W+1,7] == 1)
    
    #team can only play as many times as scheduled, IDK IF WE NEED THIS ONE BUT OTHERWISE Y-MATRIX IS ALL 1s
    @constraint(model, [i = 1:N, w = 1:W], sum(y[i,w,:]) == sum(x[i,:,w]+x[:,i,w]))

    # maximum of 1 game on day 1
    @constraint(model, [w = 1:W], sum(y[:,w,4]) <= 2) #2 teams play per game

    # maximum of 4 games on day 2
    @constraint(model, [w = 1:W], sum(y[:,w,5]) <= 8)

    # maximum of 4 games on day 3
    @constraint(model, [w = 1:W], sum(y[:,w,6]) <= 8)

    # maximum of 1 game on day 4
    @constraint(model, [w = 1:W], sum(y[:,w,7]) <= 2)


    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)


    return value.(y)
end

schedule_days (generic function with 1 method)

In [15]:
y = schedule_days(x,u,z)

Set parameter OutputFlag to value 1
Set parameter OutputFlag to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 61732 rows, 3120 columns and 69940 nonzeros
Model fingerprint: 0x73253bf4
Model has 12160 quadratic objective terms
Variable types: 0 continuous, 3120 integer (3120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 61200 rows and 1600 columns
Presolve time: 0.06s
Presolved: 12372 rows, 13360 columns, 38560 nonzeros
Variable types: 0 continuous, 13360 integer (13360 binary)

Root relaxation: objective 2.374603e+00, 2141 iterations, 0.10 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap |

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, Base.OneTo(20)
    Dimension 2, Base.OneTo(39)
    Dimension 3, 4:7
And data, a 20×39×4 Array{Float64, 3}:
[:, :, 4] =
 -0.0   1.0   1.0   1.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  0.0
  1.0   1.0   0.0  -0.0  -0.0   0.0     -0.0  -0.0  -0.0  -0.0  -0.0  0.0
 -0.0  -0.0  -0.0   0.0  -0.0  -0.0     -0.0   1.0   0.0   0.0  -0.0  0.0
  0.0  -0.0   1.0  -0.0  -0.0   0.0      0.0  -0.0  -0.0  -0.0   1.0  0.0
  0.0  -0.0   0.0   0.0   0.0  -0.0      0.0   0.0  -0.0   0.0   0.0  0.0
 -0.0   0.0  -0.0  -0.0  -0.0   0.0  …   0.0   0.0  -0.0   0.0   0.0  0.0
  0.0   0.0   0.0  -0.0   0.0  -0.0      0.0   0.0  -0.0  -0.0  -0.0  0.0
 -0.0  -0.0  -0.0   0.0   0.0  -0.0     -0.0  -0.0   0.0  -0.0   0.0  0.0
 -0.0   0.0   0.0  -0.0   0.0   0.0     -0.0  -0.0   0.0   1.0   1.0  0.0
  0.0  -0.0  -0.0   0.0   0.0   0.0     -0.0  -0.0   1.0   0.0  -0.0  0.0
 -0.0  -0.0   0.0   0.0  -0.0  -0.0  …   0.0  -0.0  -0.0  -0.0 

In [127]:
schedule_days(x,u,z)

Set parameter OutputFlag to value 1
Set parameter OutputFlag to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3972 rows, 3120 columns and 12180 nonzeros
Model fingerprint: 0xb2f1e351
Model has 12160 quadratic objective terms
Variable types: 0 continuous, 3120 integer (3120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 3440 rows and 1600 columns
Presolve time: 0.03s
Presolved: 12372 rows, 13360 columns, 38560 nonzeros
Variable types: 0 continuous, 13360 integer (13360 binary)

Root relaxation: objective 2.374603e+00, 1972 iterations, 0.06 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap |

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, Base.OneTo(20)
    Dimension 2, Base.OneTo(39)
    Dimension 3, 4:7
And data, a 20×39×4 Array{Float64, 3}:
[:, :, 4] =
  1.0   1.0   0.0   1.0   0.0  -0.0  -0.0  …   0.0   0.0  0.0   0.0  0.0  0.0
 -0.0  -0.0   0.0   0.0   0.0   0.0  -0.0      0.0   0.0  0.0   0.0  0.0  0.0
 -0.0   0.0   0.0   0.0  -0.0  -0.0  -0.0      0.0   0.0  0.0   0.0  0.0  0.0
  0.0   0.0   0.0   0.0  -0.0  -0.0   0.0      0.0   0.0  0.0   0.0  0.0  0.0
 -0.0   0.0   0.0  -0.0  -0.0  -0.0  -0.0      0.0   0.0  0.0   0.0  0.0  0.0
  1.0   0.0   0.0   0.0  -0.0  -0.0   0.0  …   1.0   1.0  1.0   0.0  0.0  0.0
  0.0  -0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0  0.0   0.0  0.0  0.0
  0.0  -0.0   0.0  -0.0   0.0   1.0   1.0      0.0   0.0  0.0   0.0  0.0  0.0
 -0.0   0.0   0.0  -0.0   1.0   1.0   0.0      0.0   1.0  0.0   0.0  0.0  0.0
  0.0  -0.0  -0.0   0.0   0.0   0.0   0.0      0.0  -0.0  0.0   0.0  0.0  0.0
  0.0   1.0   1.0   0.0

In [99]:
y[:,2,:]

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(20)
    Dimension 2, 4:7
And data, a 20×4 Matrix{Float64}:
  1.0   0.0  0.0  -0.0
 -0.0   0.0  1.0  -0.0
  0.0   0.0  0.0   1.0
  0.0   1.0  0.0   0.0
  0.0   0.0  1.0   0.0
  0.0   1.0  0.0   0.0
 -0.0   1.0  0.0   0.0
 -0.0   0.0  1.0   0.0
  0.0   0.0  1.0   0.0
 -0.0   0.0  1.0   0.0
  1.0  -0.0  0.0   0.0
  0.0   1.0  0.0  -0.0
  0.0   0.0  1.0   0.0
  0.0   1.0  0.0   0.0
  0.0   0.0  0.0   1.0
  0.0   0.0  1.0   0.0
  0.0   1.0  0.0   0.0
  0.0   1.0  0.0   0.0
  0.0   1.0  0.0   0.0
  0.0   0.0  1.0   0.0

In [110]:
x[1,6,1]

1.0